In [ ]:
!pip install tldextract

In [ ]:
from collections import Counter
import requests
import time
from urllib.parse import urlparse


import pandas as pd
import tldextract

In [ ]:
API_KEY = "API_KEY"
SEARCH_ENGINE_ID = 'SEARCH_ENGINE_ID'
query = 'dokdo.mofa.go.kr'
idx = 1
url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={idx}"

response = requests.get(url).json()

In [ ]:
#respone 어떤 키를 가지고 있는 지 확인
response.keys()

In [ ]:
# item 확인
for key in response['items'][1].keys():
    print(key, ":", response['items'][1][key])
    print("----" * 20)

In [ ]:
def response_google_cse(API_KEY, SEARCH_ENGINE_ID, query):
    idx = 1
    res = []
    url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={idx}"

    while (True):
        response = requests.get(url).json()

        if ('items' not in response.keys()):
            print("Error: Items not in response")
            break;

        len_items = len(response['items'])
        idx += len_items

        for i in range(len_items):
            res.append(response['items'][i]['link'])
        print("idx : {}".format(idx))

        if (len_items == 0):
            print("Error: Returned 0")
            break;

        time.sleep(.5)

        if (idx >  30):
            print("Error: Returned 0")
            break;
    print("Finished: {} links are collected".format(len(res)))
    return (res)

In [ ]:
def get_tld(urls):
    tlds = []
    for url in urls:
        tld = tldextract.extract(url).suffix
        if (len(tld.split('.')) > 1):
            tlds.append(tld.split('.')[-1])
        else:
            tlds.append(tld)
    return tlds

def get_domains(urls):
    domains = []
    for url in urls:
        d = "{}.{}".format(tldextract.extract(url).domain, tldextract.extract(url).suffix)
        domains.append(d)
    return domains

In [ ]:
korea_stlds = [tldextract.extract(url).suffix for url in korea]
korea_tlds = get_tld(korea)
korea_domains = get_domains(korea)

In [ ]:
koreas_stlds_count = pd.DataFrame({"tld" : dict(Counter(korea_stlds)).keys(),
                                   "value": dict(Counter(korea_stlds)).values()})
koreas_tlds_count = pd.DataFrame({"tld" : dict(Counter(korea_tlds)).keys(),
                                   "value": dict(Counter(korea_tlds)).values()})
koreas_domains_count = pd.DataFrame({"tld" : dict(Counter(korea_domains)).keys(),
                                   "value": dict(Counter(korea_domains)).values()})

In [ ]:
FILE_PATH = "FILE_PATH"
koreas_stlds_count.to_csv(os.path.join(FILE_PATH, "korea_stlds.csv"))